In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [631 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,204 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,474 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,011 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archive.ub

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
hs_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')

# Show DataFrame
hs_df.show(3)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 3 rows



In [4]:
# 2. Create a temporary view of the DataFrame.
hs_df.createOrReplaceTempView("home_sales")


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# bedrooms=4, groupby year, average price rounded to 2 decimal places
from pyspark.sql.functions import year, round, avg

# Ensure data format on date columnt
hs_df = hs_df.withColumn("date", hs_df["date"].cast("date"))

# Extract year 'date' column
hs_df = hs_df.withColumn("year", year("date"))

# Filter for houses with four bedrooms
four_bedroom_homes = hs_df.where(hs_df["bedrooms"] == 4)

# Group by year and calculate the average price, rounding to 2 decimal places
result = four_bedroom_homes.groupBy("year").agg(round(avg("price"), 2).alias("average_price"))

# Order the results by year in descending order
result = result.orderBy("year", ascending=False)

# Show the results
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# results = bedrooms=3 & bathrooms=3, average price to 2 dec places, grouped by year built

# Filter for houses with 3 beds and 3 baths
three_bed_bath_homes = hs_df.where((hs_df["bedrooms"] == 3) & (hs_df["bathrooms"] == 3))

# Group by year built and calculate the average price, rounding to 2 decimal places
result = three_bed_bath_homes.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))

# Order the results by year built in descending order
result = result.orderBy("date_built", ascending=False)

# Show the results
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Filter for houses with 3 beds, 3 baths, 2 floors and => 2000 sqft_living
three_bed_bath_2fl_homes = hs_df.where((hs_df["bedrooms"] == 3) & (hs_df["bathrooms"] == 3) & (hs_df["sqft_living"] >= 2000) & (hs_df["floors"] == 2))

# Group by year built and calculate the average price, rounding to 2 decimal places
result = three_bed_bath_2fl_homes.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))

# Order the results by year built in descending order
result = result.orderBy("date_built", ascending=False)

# Show the results
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

# Your query to find the view rating for the average price of homes >= $350,000
result = spark.sql("""
    SELECT AVG(price) AS avg_price
    FROM home_sales
    WHERE price >= 350000
""")

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------------+
|         avg_price|
+------------------+
|473796.26220224716|
+------------------+

--- 0.433488130569458 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

# Your query to find the view rating for the average price of homes >= $350,000
result = spark.sql("""
    SELECT AVG(price) AS avg_price
    FROM home_sales
    WHERE price >= 350000
""")

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------------+
|         avg_price|
+------------------+
|473796.26220224716|
+------------------+

--- 0.27755188941955566 seconds ---


In [12]:
#write the DataFrame hs_df to Parquet format with the specified mode
from google.colab import drive
drive.mount('/content/drive')
parquet_file_path = '/content/drive/My Drive/path_to_parquet_file.parquet'
hs_df.write.mode("overwrite").parquet(parquet_file_path)


Mounted at /content/drive


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
hs_df.write.partitionBy("date_built").mode("overwrite").parquet(parquet_file_path)


In [14]:
# 11. Read the parquet formatted data.
spark = SparkSession.builder.appName("ReadParquet").getOrCreate()
p_hs_df = spark.read.parquet(parquet_file_path)

In [15]:
# 12. Create a temporary table for the parquet data.
p_hs_df.createOrReplaceTempView("p_home_sales")

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()

# Your query to find the view rating for the average price of homes >= $350,000
result = spark.sql("""
    SELECT AVG(price) AS avg_price
    FROM p_home_sales
    WHERE price >= 350000
""")

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------------+
|         avg_price|
+------------------+
|473796.26220224716|
+------------------+

--- 0.7722263336181641 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("p_home_sales")

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('p_home_sales')


False